#### This file should build on what i made in v_2 and make it more in to a RL problem. I will have to define an episode, the state space and different states. The actions and the reward function has to be made as well.

In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import random
from itertools import combinations


In [2]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./one_week_collections.pkl")

In [3]:
compatible_dict = utils.get_compatibility_dict(L = L ,M = M, index = False)

One episode should be for one week. That means that I should use have hours from two days before the week starts until the saturday. 

In [4]:
def random_arg_max(possible_actions):
    imax = 0
    xmax = possible_actions[imax]  # Current maximum
    nmax = 1  # Number of maximum values at the moment

    for i in range(1, len(possible_actions)):
        if possible_actions[i] == xmax:
            nmax += 1
            if nmax * random.random() < 1.0:
                imax = i
        elif possible_actions[i] > xmax:
            nmax = 1  # Reset count since a new maximum is found
            imax = i
            xmax = possible_actions[i]  # Update the new maximum

    return imax


In [5]:
def greedy_action(Q : np.array, epsilon : float ):
    """returns the index of the greedy action

    Args:
        Q (np.array): numpy array which is sliced for the actions
        epsilon (float): float number between 0 and 1, often close to 0

    Returns:
        int: index of the greedy action
    """
    if random.random() <= (1- epsilon): # pick greedy
        return random_arg_max(Q)
    else:
        return random.randint(0, len(Q)-1) # random


In [6]:
def get_possible_dates(date : pd.Timestamp):
    """ Function to get the possible dates for placing a bid given the current date

    Args:
        date (pd.Timestamp): the current date

    Returns:
        (pd.date_range, str): the possible dates for placing a bid and for which market
    """
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")

In [7]:
def check_constraints_for_hour(possible_assets : [[new_meters.PowerMeter]], hour : pd.Timestamp, possible_volume : float, expected_price : float, market : final_markets.ReserveMarket):
            
    max_vol = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]  # set of volumes for markets
    constrained_vol = possible_volume if possible_volume >= market.min_volume and possible_volume < max_vol else 0
    # get the prices for the given market within the given hours
    possible_price = market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]  #price for markets
    # Calculate the possible revenues
    """print(f"possible_prices in function : {possible_prices}")
    print(f"expected_prices in function : {expected_prices}")"""
    bids_to_be_made = possible_price if possible_price >= expected_price else 0
    """print(f"bids_to_be_made in function : {bids_to_be_made}")
    print(f"possible_volumes in function : {constrained_vols}")"""
    possible_revenue = constrained_vol * bids_to_be_made
    
    #print(f"possible_revenues in function : {possible_revenues}")
    possible_assets = possible_assets if possible_revenue > 0 else [] 
    #print(f"possible_assets in function : {possible_assets}")
    return possible_revenue, possible_assets

In [8]:
def place_hourly_bids(hour : pd.Timestamp, expected_price : float, available_assets : dict, market : final_markets.ReserveMarket):
    """ Function to place bids for a given market and set of hours. The bids are placed for every hour in the set of hours. The bids are placed for the assets that are not already bid in to other markets in the given hours. The bids are placed for the assets that are compatible with the given market.

    Args:
        possible_hours (pd.Timestamp]): The time stamps for which the bids are placed
        available_assets (dict): Dictionary with the available assets for each hour
        market (final_markets.ReserveMarket): the market to be bid in to

    Returns:
        tuple (np.array, np.array): The possible revenue of the placed bids for each hour and the assets which are bid for each hour
    """
    #hourly_assets = [available_assets[hour] for hour in possible_hours]
    possible_assets = [asset for asset in available_assets if asset in compatible_dict[market]]
    if market.direction == "up":
        # check the compatibility for the assets
        possible_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets])
        #print(f"possible_volumes: {possible_volumes}")
        possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour ,possible_volume,  expected_price , market)
    elif market.direction == "down":
        possible_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets])
        #print(f"possible_volumes: {possible_volumes}")

        possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour, possible_volume, expected_price, market)
    else:        
        possible_up_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets])
        possible_down_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets])
        #print(f"possible_up_volumes: {possible_up_volumes}")
        #print(f"possible_down_volumes: {possible_down_volumes}")
        # possible volumes should be != 0 if both up and down volume is higher than min_volume. If both vols are higher than min_volume, then the actuale volume should be decided by the lowest one
        actual_volumes = min(possible_up_volume, possible_down_volume)
        #print(f"actual_volumes: {actual_volumes}")
        # Find the hours where both up and down volume is higher than min_volume
        possible_volume = actual_volumes if possible_up_volume >= market.min_volume and possible_down_volume >= market.min_volume else 0 
        #possible_volumes = np.where((possible_up_volumes >= min_vols and possible_down_volumes >= min_vols), actual_volumes, 0) 
        #print(f"possible_volumes: {possible_volumes}")
        possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour, possible_volume, expected_price, market)
        """print(f"possible_revenues: {possible_revenues}")
        print(f"possible_assets: {possible_assets}")"""
    return (possible_revenue, possible_assets)
        


In [11]:
def get_portfolios_for_market(possible_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket, hour : pd.Timestamp):
     # Initialize a list to store feasible combinations
    feasible_combinations = []

    # Check all possible combinations
    for r in range(1, len(possible_assets) + 1):
        for combination in combinations(possible_assets, r):
            if len(combination) > 100:
                total_volume = sum(asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in combination)
                if total_volume >= market.min_volume:
                    feasible_combinations.append(combination)
            else:
                continue
    
    return feasible_combinations

In [12]:
m = M[23]
p = get_portfolios_for_market([asset for asset in L if asset in compatible_dict[m]], m, H[35])

In [ ]:


def get_all_feasible_portfolios(M : [final_markets.ReserveMarket], L : [new_meters.PowerMeter], H : [pd.Timestamp]):
    
    portfolios = {}
    for market in M:
        possible_assets = [asset for asset in L if asset in compatible_dict[market]]
        for hour in H:
            if market.direction == "up":
                # check the compatibility for the assets
                possible_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets])
                if possible_volume < market.min_volume:
                    possible_assets = []
                else:
                    # check all possible combination of assets and see if the total volume of the assets in the combination is higher than min_volume
                    feasible_combinations = get_portfolios_for_market(possible_assets, market, hour)
                    # Add feasible combinations to the portfolio dictionary
                    portfolios[(market, hour)] = feasible_combinations
                    
                
                #print(f"possible_volumes: {possible_volumes}")
                possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour ,possible_volume , market)
            elif market.direction == "down":
                possible_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets])
                #print(f"possible_volumes: {possible_volumes}")

                possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour, possible_volume, market)
            else:        
                possible_up_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets])
                possible_down_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets])
                #print(f"possible_up_volumes: {possible_up_volumes}")
                #print(f"possible_down_volumes: {possible_down_volumes}")
                # possible volumes should be != 0 if both up and down volume is higher than min_volume. If both vols are higher than min_volume, then the actuale volume should be decided by the lowest one
                actual_volumes = min(possible_up_volume, possible_down_volume)
                #print(f"actual_volumes: {actual_volumes}")
                # Find the hours where both up and down volume is higher than min_volume
                possible_volume = actual_volumes if possible_up_volume >= market.min_volume and possible_down_volume >= market.min_volume else 0 
                #possible_volumes = np.where((possible_up_volumes >= min_vols and possible_down_volumes >= min_vols), actual_volumes, 0) 
                #print(f"possible_volumes: {possible_volumes}")
                possible_revenue, possible_assets = check_constraints_for_hour(possible_assets, hour, possible_volume, market)
                """print(f"possible_revenues: {possible_revenues}")
                print(f"possible_assets: {possible_assets}")"""

    
    return portfolios

In [ ]:
def epsilon_greedy(Q : np.array, epsilon : float, market : final_markets.ReserveMarket, available_assets : dict, hour : pd.Timestamp):
    """returns the index of the greedy action

    Args:
        Q (np.array): numpy array which is sliced for the actions
        epsilon (float): float number between 0 and 1, often close to 0

    Returns:
        int: index of the greedy action
    """
    # will find all possible feasible portfolios of assets to bid in to the market
    rev, assets = place_hourly_bids(hour, Q[0], available_assets, market)
        
    if random.random() <= (1- epsilon): # pick greedy
        return random_arg_max(Q)
    else:
        return random.randint(0, len(Q)-1) # random

In [126]:
def Sarsa(epsilon, alpha, num_episodes, L, M, H):
    
    revenues = {}
    bid_timeframe = H[24:] # the hours where bids can be placed in
    place_bid_hours = [hour for hour in H[:-48] if hour.hour == 7 or hour.hour == 17 or hour.hour == 18] # the hours where bids can be placed from

    sup_market_names = ["FCR", "aFRR"]
    markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
    market_names = [market.name for market in markets]
    # will only use FCR and aFRR
    
    available_assets = {hour: L.copy() for hour in bid_timeframe} 
    bids = {(market.name, hour): [] for hour in bid_timeframe for market in markets}
    
    Q = np.zeros((7, 24, len(L)+1, len(market_names)+1 )) # day of week, hour of day, available assets, possible markets, assets already bid
    (possible_hours, market_name) = get_possible_dates(place_bid_hours[0])
    
    "an action should be to bid in to on of the possible markets returned from get_possible_dates"
    # will have to make a slice of Q where only the indexes for the possible markets are included
    possible_markets = [m for m in markets if market_name in m.name]
    indexes = [market_names.index(m.name) for m in possible_markets]
    q_0 = Q[possible_hours[0].weekday(), possible_hours[0].hour, :, indexes]

    
    action_0 = greedy_action(q_0, epsilon, )
    value_0 = Q[possible_hours[0].weekday(), possible_hours[0].hour, len(available_assets[possible_hours[0]]), action_0]
    # extract hours from H where hour == 7, 17, 18
    expected_prices = {}
    for directions in ["up", "down", "both"]:
        for area in ["NO1", "NO2", "NO3", "NO4", "NO5"]:
            for hour in bid_timeframe:
                expected_prices[(directions, area, hour)] = np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])
            
    for episode_n in range(num_episodes):
        if episode_n > 0:
            epsilon -= 0.02
            alpha -= 0.001
        revenue = 0
        bids = {(market.name, hour): [] for hour in bid_timeframe for market in markets}
        available_assets = {hour: L.copy() for hour in bid_timeframe}
        for place_hour in place_bid_hours:
            (possible_hours, market_name) = get_possible_dates(place_hour)
            if len(possible_hours) != 24:
                #print(f"No bids for {hour}")
                #print(f"possible_hours: {len(possible_hours)}")
                continue
            possible_markets = [m for m in markets if market_name in m.name]
            indexes = [market_names.index(m.name) for m in possible_markets]
            for market in possible_markets: # This will turn in to a problem as it should be possible to place several bids for each hour, but the Q-table is only updated once (for one market) for each hour
                # if i continue to do it like this, i must change the action to not choose the market, but for each market for each hour choose the number of assets to bid
                # maybe ill change the actions to rather be the number of assets to bid. If I do this, I must be secure that the number of assets to bid still holds all of the constraints.
                    # If I do this, i should change the espilon-greedy function to change between the valid actions for each hour for each market
                    # The epsilon greedy function must find each combination of feasible number of assets and to each market for each hour. Maybe I should precompute this and store it in a dictionary
                for h, bid_hour in enumerate(possible_hours):
                    (possible_revenues, possible_assets) = place_hourly_bids(market=market, available_assets=available_assets[bid_hour], hour=bid_hour, expected_price=expected_prices[h])
                    revenue += np.sum(possible_revenues)
                    # Select assets with positive revenue for bidding
                    assets_to_bid = [asset for asset in possible_assets if possible_revenues > 0]
                    # Flatten the list if it's a list of lists
                    #assets_to_bid = [item for sublist in assets_to_bid for item in sublist]
                    # Store the bid information
                    bids[(market.name, bid_hour)] = [asset.meter_id for asset in assets_to_bid]
                    # Update available assets
                    available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in assets_to_bid]
                    q_1 = Q[bid_hour.weekday(), bid_hour.hour, len(available_assets[bid_hour]), indexes]
                    action_1 = greedy_action(q_1, epsilon)
                    value_1 = Q[bid_hour.weekday(), bid_hour.hour, len(available_assets[bid_hour]), action_1]
                    Q[bid_hour.weekday(), bid_hour.hour, len(available_assets[bid_hour]), action_0] = value_0 + alpha * (possible_revenues + value_1 - value_0)
                    action_0, value_0 = action_1, value_1
        revenues[episode_n] = revenue
    return bids, revenues, available_assets, Q
    
    
    

In [127]:
b, r, a, Q = Sarsa(epsilon=0.3, alpha =  0.1, num_episodes = 10, L =  L, M= M,  H= H)

In [129]:
r.values()

dict_values([40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999, 40162.09030041999])

In [131]:
Q

array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.17900555e-01, 0.00000000e+00, 1.04029275e+02, ...,
          3.23435848e+01, 5.69633663e+00, 2.32974013e+00]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        

In [132]:
def feasibility_check(bids, L,M):
    """ Function to check if the bids are feasible. The bids are feasible if there are no duplicates in the bids for each hour."""
    markets= set([market[0] for market in bids.keys()])
    #print(f"Markets: {markets}")
    hours = set([market[1] for market in bids.keys()])
    #print(f"Hours: {hours}")
    keys = list(bids.keys())
    #print(f"Keys: {keys}")
    hourly_assets = {}
    asset_meter_id_dict = {asset.meter_id: asset for asset in L}
    asset_info = {}
    markets_name_dict = {market.name: market for market in M}

    for hour in hours:
       # print(f"Hour: {hour}")
        hourly_as_list = []
        for market_name in markets:
            #print(f"Market: {market}")
            if (market_name, hour) in keys:
                hourly_as_list.append(bids[(market_name, hour)])
                asset_ids_in_bid = bids[(market_name, hour)]
                market = markets_name_dict[market_name]
                assets_in_bid = [asset_meter_id_dict[asset] for asset in asset_ids_in_bid]
                if len(assets_in_bid) > 0:
                    assert len(assets_in_bid) == len(set(assets_in_bid)), f"Duplicate assets in hour {hour}"
                    if market.direction == "up":
                        assert all([asset.direction != "down" for asset in assets_in_bid]), f"Down assets in up market {market.name} in hour {hour}"
                        total_flex_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in assets_in_bid])
                    elif market.direction == "down":
                        assert all([asset.direction != "up" for asset in assets_in_bid]), f"Up assets in down market in hour {hour}"
                        total_flex_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in assets_in_bid])
                    else:
                        if dominant_directions[H.get_loc(hour)] == "up":
                            #assert all([asset.direction != "down" for asset in assets]), f"Down assets in up market in hour {hour}"
                            total_flex_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in assets_in_bid])
                        elif dominant_directions[H.get_loc(hour)] == "down":
                            #assert all([asset.direction != "up" for asset in assets]), f"Up assets in down market in hour {hour}"
                            total_flex_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in assets_in_bid])
                    assert total_flex_volume >= market.min_volume, f"Total flex volume {total_flex_volume} is less than the minimum volume {market.min_volume} in hour {hour} for market {market.name}"
                    total_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
                    assert total_flex_volume <= total_max_volume, f"Total flex volume {total_flex_volume} is more than the maximum volume {total_max_volume} in hour {hour}"
                    assert all([asset.response_time <= market.response_time for asset in assets_in_bid]), f"Asset with response time higher than the market {market.name} response time in hour {hour}"
                    asset_info[(market_name, hour)] = (total_flex_volume, len(assets_in_bid)) 
        # Add the flattened list to the dictionary
        hourly_assets[hour] = [item for sublist in hourly_as_list for item in sublist]
    
    df = pd.DataFrame(columns=["Hour", "Market", "Total Flex Volume assigned", "Number of assets assigned"])
    #print(list(asset_info.keys()))
    #print(hourly_assets)
    for hour in hours:
        if hourly_assets[hour] == []:
            continue
        else:
            """print(f"Hour : {hour}")
            print(f"type of hour object : {type(hour)}")
            print(f"Hourly assets : {hourly_assets[hour]}")
            print(f"length of hourly assets : {len(hourly_assets[hour])}")"""
            # check if there are duplicates
            if len(hourly_assets[hour]) != len(set(hourly_assets[hour])) and hourly_assets[hour] != []:
                print(f"Duplicate assets in hour {hour}")
                # print the duplicates
                print(f"length of hourly assets : {len(hourly_assets[hour])}")
                print(f"length of set of hourly assets : {len(set(hourly_assets[hour]))}")
                return False
        for market in markets:
            if (market, hour) in list(asset_info.keys()):
                df.loc[len(df)] = [hour, market, asset_info[(market, hour)][0], asset_info[(market, hour)][1]]               
    return hourly_assets, df.sort_values(by=["Hour", "Market"])
    

In [133]:
ha, df = feasibility_check(b, L, M)

In [121]:
df

,Hour,Market,Total Flex Volume assigned,Number of assets assigned
366,2023-06-15 00:00:00+02:00,aFRR up_NO5,3.384268,889
531,2023-06-15 05:00:00+02:00,aFRR up_NO1,1.226750,772
532,2023-06-15 05:00:00+02:00,aFRR up_NO5,2.256698,889
653,2023-06-15 06:00:00+02:00,aFRR up_NO1,1.034972,772
654,2023-06-15 06:00:00+02:00,aFRR up_NO5,2.183059,889
...,...,...,...,...
689,2023-06-30 20:00:00+02:00,FCR_N_D_2_NO2,1.334489,127
688,2023-06-30 20:00:00+02:00,FCR_N_D_2_NO5,3.053724,495
807,2023-06-30 21:00:00+02:00,FCR_N_D_2_NO5,1.800752,495
673,2023-06-30 22:00:00+02:00,FCR_N_D_2_NO5,1.749646,495
